In [1]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 6.5 MB/s eta 0:00:0000:0100:01m


In [2]:
!git clone https://github.com/open-mmlab/mmclassification.git
%cd mmclassification/

# Install MMClassification from source
!pip install -e . 

Cloning into 'mmclassification'...
remote: Enumerating objects: 10262, done.
remote: Total 10262 (delta 0), reused 0 (delta 0), pack-reused 10262
Receiving objects: 100% (10262/10262), 10.43 MiB | 12.84 MiB/s, done.
Resolving deltas: 100% (7143/7143), done.
/kaggle/working/mmclassification
Obtaining file:///kaggle/working/mmclassification
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmcls


# Imports

In [3]:
import mmcls
import torch, torchvision
import numpy as np
import pandas as pd
import os
import random
from shutil import copyfile
#import pydicom as dicom
import cv2
from pathlib import Path
import wandb
from kaggle_secrets import UserSecretsClient
import json
import wandb
import mmcv
from mmcls.apis import inference_model, init_model, show_result_pyplot
from mmcv import Config
import time
import mmcv
import os.path as osp

from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

/opt/conda/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


In [4]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb-key") 
wandb.login(key=wandb_api)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Preparing the Dataset

In [5]:
BASE_PATH = '/kaggle/input'
HACKATHON_PATH = os.path.join(BASE_PATH,'hackathon')
SIIM_DETECTION_PATH = os.path.join(BASE_PATH,'siim-covid19-detection')
PAPER_IMAGES_PATH = os.path.join(BASE_PATH,'paperoriginal','CroppedSegmentedImages')
COVID_QU_EX_PATH = os.path.join(BASE_PATH,'maskedcovidquexdataset')

train = []
test = []
test_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
train_count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
filename_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
path_label = {'normal': [], 'pneumonia': [], 'COVID-19': []}
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
mapping = {0:'normal',1:'COVID-19',2:'pneumonia'}
inverse_mapping = {'normal':0,'COVID-19':1,'pneumonia':2}
used_datasets_classification= []

In [6]:
for j,directory in enumerate(os.listdir(PAPER_IMAGES_PATH)):
    if not directory.startswith('.') and not directory=='CRXNIH':
        for i,file in enumerate(os.listdir(os.path.join(PAPER_IMAGES_PATH,directory))):
            try:
                cat = int(file.split('__')[1])
            except Exception as e:
                cat = int(file.split('_')[1])
            finally:
                count[mapping[cat]] += 1
                entry = [int(count[mapping[cat]]),'paperoriginal'+ '/'+ 'CroppedSegmentedImages' + '/'+ directory + '/' + file,str(cat)]
                filename_label[mapping[cat]].append(entry)
    used_datasets_classification.append(f'paper_{directory}')
print(count)

{'normal': 60594, 'pneumonia': 22746, 'COVID-19': 8014}


for i,parent_directory in enumerate(os.listdir(COVID_QU_EX_PATH)):
    if not parent_directory.startswith('.'):
        full_parent_directory = os.path.join(COVID_QU_EX_PATH,parent_directory)
        for j,category in enumerate(os.listdir(full_parent_directory)):
            if not category.startswith('.'):
                full_category_directory = os.path.join(full_parent_directory,category)
                for k,img_dir in enumerate(os.listdir(full_category_directory)):
                    if not img_dir.startswith('.'):
                        img_dir = img_dir.replace(' ','_')
                        count[category] += 1
                        entry = [int(count[category]) ,os.path.join('maskedcovidquexdataset',parent_directory,category,img_dir)
                        , str(inverse_mapping[category])]
                        filename_label[category].append(entry)
print(count)
used_datasets_classification.append('maskedcovidquexdataset')

In [7]:
np.random.seed(seed=2)
test_per = 0.1
for key in filename_label.keys():
    if key == 'pneumonia':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    elif key == 'COVID-19':
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    else:
        test_patients = np.random.permutation(count[key])[:int(count[key]*test_per)]
    for i,patient in enumerate(filename_label[key]):
        if int(patient[0]) in test_patients:
            test.append(patient)
            test_count[key] += 1
        else:
            train.append(patient)
            train_count[key] += 1

print('test count: ', test_count)
print('train count: ', train_count)

test count:  {'normal': 6058, 'pneumonia': 2274, 'COVID-19': 800}
train count:  {'normal': 54536, 'pneumonia': 20472, 'COVID-19': 7214}


In [8]:
txt_name = 'kaggle_txt_sept'

In [9]:
WORK_DIR = '/kaggle/working/Alvaro-TFM'


train_file = open(os.path.join(f'/kaggle/working/{txt_name}_train_split.txt'),"w")
for sample in train:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    train_file.write(info)

train_file.close()

test_file = open(os.path.join(f'/kaggle/working/{txt_name}_test_split.txt'), "w")
for sample in test:
    info = str(sample[0]) + ' ' + sample[1] + ' ' + sample[2] + '\n'
    test_file.write(info)
test_file.close()

In [10]:
train_ann_file = '/kaggle/working/train_ann_file.txt'
val_ann_file = '/kaggle/working/val_ann_file.txt'

train_file = open(train_ann_file,"w")
for sample in train:
    info = sample[1] + ' ' + sample[2] + '\n'
    train_file.write(info)

train_file.close()

val_file = open(val_ann_file, "w")
for sample in test:
    info = sample[1] + ' ' + sample[2] + '\n'
    val_file.write(info)
val_file.close()

In [12]:
train_split_file = f'/kaggle/working/{txt_name}_train_split.txt'
test_split_file = f'/kaggle/working/{txt_name}_test_split.txt'


In [13]:
print(train_split_file)
print(test_split_file)

/kaggle/working/kaggle_txt_sept_train_split.txt
/kaggle/working/kaggle_txt_sept_test_split.txt


# Model Definition

In [14]:
cfg_path = '/kaggle/working/mmclassification/configs/resnext/resnext101_32x4d_b32x8_imagenet.py'
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/resnext/resnext101_32x4d_b32x8_imagenet_20210506-e0fa3dd5.pth'

In [15]:
BATCH_SIZE = 64
EPOCHS = 12
LEARNING_RATE = 0.005


cfg = Config.fromfile(cfg_path)
cfg.model.head.num_classes = 3
cfg.model.head.topk = (1, )

cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')

cfg.data.samples_per_gpu = BATCH_SIZE
cfg.data.workers_per_gpu = 2

classes = ['normal','COVID-19','pneumonia']
dataset_type = 'CustomDataset'

cfg.data.train.data_prefix = '/kaggle/input'
cfg.data.train.ann_file = train_ann_file
cfg.data.train.classes = classes

# Specify the path and meta files of validation dataset
cfg.data.val.data_prefix = '/kaggle/input'
cfg.data.val.ann_file = val_ann_file
cfg.data.val.classes = classes

cfg.evaluation['metric_options']={'topk': (1, )}

# Specify the optimizer
cfg.optimizer = dict(type='SGD', lr=LEARNING_RATE, momentum=0.9, weight_decay=0.0001)
cfg.optimizer_config = dict(grad_clip=None)

# Specify the learning rate scheduler
cfg.lr_config = dict(policy='step', step=1, gamma=0.1)
cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)
cfg.gpu_ids = [0]
cfg.device = 'cuda'

In [16]:
cfg.data.train = dict(type=dataset_type,
                     data_prefix='/kaggle/input',
                     ann_file=train_ann_file,
                     classes=classes,
                     pipeline=cfg.train_pipeline)
cfg.data.val = dict(type=dataset_type,
                     data_prefix='/kaggle/input',
                     ann_file=val_ann_file,
                     classes=classes,
                     pipeline=cfg.test_pipeline)

In [17]:
work_dir = '/kaggle/working/resnext'
os.makedirs(work_dir)


In [18]:
cfg.work_dir = work_dir

In [20]:
import pytz
from datetime import datetime
timezone = pytz.timezone('Europe/Madrid')
now = datetime.now(timezone)

dt_string = now.strftime("%d/%m_%H:%M")

In [21]:
PROJECT_NAME = 'Final-Covid19-Detection'
NOTEBOOK_NAME = 'TFM - Classification pretraining'
EXP_NAME = 'ResNext101'

cfg.log_config.hooks = [dict(type='TextLoggerHook'),
                        dict(type='MMClsWandbHook',
                             init_kwargs=dict(project=PROJECT_NAME,
                                              name=f'{EXP_NAME}_{dt_string}',
                                              job_type='model_train',
                                              entity='alvaromoureupm',
                                             notes=f'Notebook: {NOTEBOOK_NAME}',
                                             config=cfg._cfg_dict.to_dict()),
                             log_artifact=True,
                             interval = 100
                            )
                       ]

# Train model

In [22]:
model = build_classifier(cfg.model)
model.init_weights

<bound method BaseModule.init_weights of ImageClassifier(
  (backbone): ResNeXt(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [23]:
datasets = [build_dataset(cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

In [ ]:
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()))

2022-10-07 09:54:31,437 - mmcls - INFO - Start running, host: root@0e30411c9dca, work_dir: /kaggle/working/resnext
2022-10-07 09:54:31,439 - mmcls - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMClsWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMClsWandbHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
 -------------------- 
after_train_it